In [ ]:
!pip install langchain-openai
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict , Literal
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel , Field

In [ ]:
load_dotenv()

False

In [ ]:
model =  ChatOpenAI(model='gpt-4o-mini')

In [ ]:
class sentiment_schema(BaseModel):
  sentiment: Literal['Postive' , 'Negtive'] =  Field(description='sentiment of the review')

In [ ]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [ ]:
structured_model = model.with_structured_output(sentiment_schema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [ ]:
prompt = f'What is the Sentiment of the following review - The software is too bad'

In [ ]:
structured_model.invoke(prompt)

sentiment_schema(sentiment='Negtive')

In [ ]:
class ReviewState(TypedDict):
  review: str
  sentiment: Literal['Postive' , 'Negtive']
  diagnosis: dict
  responce: str


In [ ]:
graph = StateGraph(ReviewState)

In [ ]:
def find_sentiment(state: ReviewState):
    prompt = f"For the following review find out the sentiment\n{state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    return {'sentiment':sentiment}

In [ ]:
def check_sentiment(state: ReviewState) -> Literal["postive_responce" , "run_diagnosis"]:
  if state['sentiment'] == 'Postive':
    return 'postive_responce'
  else:
    return 'run_diagnosis'

In [ ]:
def postive_responce(state: ReviewState):
  prompt = f'''
  Write a warm thank-you messaage to the customer for the following review \n\n
  {state['review']}\n
  Also, Kindly ask the user to leave the feedback on the website.
  '''

  responce = model.invoke(prompt)
  return {'responce':responce}

In [ ]:
def run_diagnosis(state: ReviewState):
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response = structured_model2.invoke(prompt)

    return {'diagnosis': response.model_dump()}

In [ ]:
def negative_response(state: ReviewState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = model.invoke(prompt).content

    return {'response': response}

In [ ]:
graph.add_node('find_sentiment',find_sentiment)
graph.add_node('postive_responce',postive_responce)
graph.add_node('run_diagnosis',run_diagnosis)
graph.add_node('negative_response',negative_response)




graph.add_edge(START,'find_sentiment')
graph.add_conditional_edges('find_sentiment' ,  check_sentiment)
graph.add_edge('postive_responce',END)
graph.add_edge('run_diagnosis','negative_response')
graph.add_edge('negative_response',END)



In [ ]:
workflow = graph.compile()

In [ ]:
intial_state = {'review':'The software is too bad'}

In [ ]:
workflow.invoke(intial_state)

{'review': 'The software is too bad',
 'sentiment': 'Negtive',
 'diagnosis': {'issue_type': 'Other',
  'tone': 'frustrated',
  'urgency': 'medium'}}